In [75]:
import torch
import tqdm

## Compare

In [89]:
model = 'activations_pythia_160m/wikidata_occupation/model-activations/EleutherAI_pythia-160m_2L_1.pt'
sae = 'activations_pythia_160m/wikidata_occupation/sae-activations/EleutherAI_sae-pythia-160m-32k_2L_1.pt'


checkpoint_model = torch.load(model, map_location=torch.device('cuda:1'), weights_only=False)
print(len(checkpoint_model))
print(type(checkpoint_model))
print(checkpoint_model[3])
print(checkpoint_model[4])
print(checkpoint_model[5])

print('_'*25)
checkpoint_sae = torch.load(sae, map_location=torch.device('cuda:1'), weights_only=False)
print(len(checkpoint_sae))
print(type(checkpoint_sae))
print(checkpoint_sae[3])
print(checkpoint_sae[4])
print(checkpoint_sae[5])

print('_'*25)
# Tallies?
assert len(checkpoint_sae) == len(checkpoint_model), "Lengths dont match"
for i in tqdm.tqdm(range(len(checkpoint_sae))):
    assert checkpoint_sae[i].shape[0] == checkpoint_sae[i].shape[0], f"Positions dont match for index {i}"
    non_zero = (checkpoint_sae[i] != 0).sum(dim = -1)
    assert non_zero.max() == 32, "More than 32 non zeros"
    assert non_zero.min() == 32, "Less than 32 non zeros"

1024
<class 'list'>
tensor([[-0.0637,  0.3579, -1.2098,  ..., -0.3583,  1.1180, -0.9611],
        [-0.0494, -0.1564,  0.4800,  ..., -0.3649,  0.4646, -0.3144],
        [-0.2584, -0.5969,  0.3574,  ..., -0.5000,  0.1538, -0.2014],
        [-0.0246, -0.7495, -0.0599,  ...,  0.5706,  0.3490, -0.3277],
        [ 0.0792, -1.0952,  0.8255,  ..., -0.1914,  0.7000,  0.5294],
        [ 0.1731, -0.2657,  0.6635,  ..., -0.6045,  1.4742, -0.2044]],
       device='cuda:1')
tensor([[-0.9551,  0.3997, -0.2352,  ..., -0.3154,  0.8510, -0.3169],
        [-0.5009, -0.0760, -0.5180,  ..., -0.2876,  0.2634,  0.4550]],
       device='cuda:1')
tensor([[-0.6538, -0.1996,  0.0170,  ..., -0.3735,  1.1986, -0.3299],
        [-0.0495, -0.6130,  1.4299,  ..., -0.5024,  0.1963,  0.1940],
        [-1.4498, -0.5782, -0.4229,  ..., -1.0788,  0.7377, -0.4139]],
       device='cuda:1')
_________________________
1024
<class 'list'>
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [

100%|██████████| 1024/1024 [00:00<00:00, 8804.93it/s]
